In [1]:
import MDAnalysis as mda
import numpy as np
import pandas as pd
import sys
#sys.path.append('/Users/dburns/Library/CloudStorage/Box-Box/my_scripts')
from ChACRA.ContactAnalysis.ContactFrequencies import *
import os

In [3]:
#u = mda.Universe('/Users/dburns/Library/CloudStorage/Box-Box/1_GROUP_data/Enzymes/atcase/from_pronto/1q95/1q95_minimized_nohoh.pdb')
u = mda.Universe('../../../from_box/atcase/1q95_minimized_nohoh.pdb')
protein = u.select_atoms('protein')
u = mda.Merge(protein)
universe = u.copy()
residues = {seg.segid: seg.residues.resnames for seg in universe.segments}
segids = list(residues.keys())
# make a square matrix that will be filled with True values for identical subunits
array = np.zeros((len(segids),len(segids)),dtype=np.bool_)
# every subunit is identical with itself
np.fill_diagonal(array,True)
# work with it as a df
identical_table = pd.DataFrame(array, columns=segids, index=segids)

#contact_dir = '/Users/dburns/Library/CloudStorage/Box-Box/1_GROUP_data/Enzymes/atcase/from_nova/contacts/freqs'
contact_dir = '../../../from_box/atcase/freqs'
contact_files = [f'{contact_dir}/{file}' for file in os.listdir(contact_dir)]
contact_files.sort()
cont = make_contact_frequency_dictionary(contact_files)
cont = ContactFrequencies(pd.DataFrame(cont))

In [4]:
identical_subunits = find_identical_subunits(u)

In [6]:
sorted_distances = get_chain_distances(identical_subunits, u)

In [7]:
sorted_distances

{0: {('A', 'C'): 1.7065504023539435,
  ('A', 'B'): 1.7316963647164045,
  ('A', 'E'): 2.276290121014368,
  ('A', 'D'): 14.695149666859432,
  ('A', 'F'): 23.72274977485476},
 1: {('G', 'J'): 1.7411957335158068,
  ('G', 'I'): 27.48132477068823,
  ('G', 'H'): 28.549628162117425,
  ('G', 'K'): 30.40205853744378,
  ('G', 'L'): 44.52822064542099},
 (0, 1): {('A', 'G'): 1.6783015165014643,
  ('A', 'H'): 2.3374572559516813,
  ('A', 'K'): 10.352570156109394,
  ('A', 'I'): 11.368680308402194,
  ('A', 'J'): 19.089641017069628,
  ('A', 'L'): 27.828149816818694}}

In [8]:
sorted_all_chain_dists =  get_all_chain_dists(u)

In [9]:
sorted_all_chain_dists

{'F': {'L': 1.6868845940431783,
  'E': 1.6997656953686828,
  'D': 1.7251738858528087,
  'B': 1.8194105875476971,
  'K': 2.539416945533425,
  'H': 11.278661873099622,
  'J': 12.173696860953113,
  'C': 14.49427711966249,
  'I': 16.762685420723876,
  'A': 23.72274977485476,
  'G': 32.02890115513436},
 'B': {'C': 1.6813929744436025,
  'H': 1.6994563762386845,
  'A': 1.7316963647164045,
  'F': 1.8194105875476971,
  'I': 2.405174661796874,
  'L': 10.596713522639728,
  'G': 11.396500598235063,
  'E': 12.988872906635818,
  'K': 16.547568071963777,
  'D': 24.546360514064247,
  'J': 30.74457695856177},
 'A': {'G': 1.6783015165014643,
  'C': 1.7065504023539435,
  'B': 1.7316963647164045,
  'E': 2.276290121014368,
  'H': 2.3374572559516813,
  'K': 10.352570156109394,
  'I': 11.368680308402194,
  'D': 14.695149666859432,
  'J': 19.089641017069628,
  'F': 23.72274977485476,
  'L': 27.828149816818694},
 'I': {'C': 1.649130675274525,
  'L': 1.8766633250333378,
  'B': 2.405174661796874,
  'D': 10.76695

# calculate the distances between each chain and all others

### Use contact df to determine which chains actually make contact

In [18]:

df = cont.freqs
partner_chains = get_contacting_chains(df)

In [19]:
partner_chains

{'B': {'A', 'B', 'C', 'E', 'F', 'H', 'I'},
 'C': {'A', 'B', 'C', 'D', 'F', 'G', 'I', 'J'},
 'D': {'C', 'D', 'E', 'F', 'I', 'J', 'L'},
 'E': {'A', 'B', 'D', 'E', 'F', 'J', 'K'},
 'A': {'A', 'B', 'C', 'E', 'G', 'H'},
 'L': {'D', 'F', 'I', 'L'},
 'J': {'C', 'D', 'E', 'G', 'J'},
 'I': {'B', 'C', 'D', 'I', 'L'},
 'G': {'A', 'C', 'G', 'J'},
 'H': {'A', 'B', 'H', 'K'},
 'F': {'B', 'C', 'D', 'E', 'F', 'K', 'L'},
 'K': {'E', 'F', 'H', 'K'}}

# New Averaging Loop

In [94]:
def asymmetric_measurements(seg_combo, identical_subunits, u, all_chain_dists):
    '''
    This will identify all the equivalent interaction pairs as seg_combo
    all_chain_dists is included so the best residue is used to create the best variance in the measured distances 
    
    seg_combo : tuple(string, string)
        The pair of chains that you want to identify all the equivalent interacting pairs for.
        ('A','B') will find other identical chains pairs that interact with the same geometry as A and B
    
    identical_subunits : dictionary
        The dictionary with integer values and lists of identical subunits.

    all_chain_dists : dictionary
        The dictionary with chain_id values and nest dictionaries giving the chain keys and the minimum distance to them as values
    
    Returns
    -------
    list of tuples
    each tuple will be a pair of chains with equivalent interaction geometry
    
    

    # whole protein center of mass
    This will identify all the equivalent interaction pairs as seg_combo
    all_chain_dists is included so the best residue is used to create the best variance in the measured distances 
    
    seg_combo : tuple(string, string)
        The pair of chains that you want to identify all the equivalent interacting pairs for.
        ('A','B') will find other identical chains pairs that interact with the same geometry as A and B
    
    identical_subunits : dictionary
        The dictionary with integer values and lists of identical subunits.

    all_chain_dists : dictionary
        The dictionary with chain_id values and nest dictionaries giving the chain keys and the minimum distance to them as values
    
    Returns
    -------
    list of tuples
    each tuple will be a pair of chains with equivalent interaction geometry
    
    '''
    com = u.select_atoms('protein').center_of_mass()
    # indices of atoms corresponding to chain seg_combo[0]
    A = np.where(u.atoms.segids == f'{seg_combo[0]}')[0]
    # asymmetric center of mass that is some point on the periphery of seg_combo[0] near seg_combo[1]
    # if they're not within 5 angstroms, this won't work - need to just identify closest chain, pick one at random
    # and then establish the as_com with that
    neighbor = find_best_asymmetric_point(u, seg_combo[0], all_chain_dists)
    as_com = u.select_atoms(f'(around 5 chainID {neighbor}) and chainID {seg_combo[0]}').center_of_mass()
    # the distances between all the atoms in seg_combo[0] and the asymmetric center of mass
    dists = np.apply_along_axis(np.linalg.norm, 1,u.atoms[A].positions - as_com)
    #find the closest res to the as_com so that this residue can be used for the reference point for other identical chains
    resid = u.atoms[A][np.where(dists==dists.min())[0][0]].resid
    # better to just use CA instead of this atom for distance measurements since there's more variability in sidechain positions
    atom = u.atoms[A][np.where(dists==dists.min())[0][0]].name
    A_pos = u.select_atoms(f'chainID {seg_combo[0]} and resnum {resid} and name CA').positions[0]
    # center of mass of seg_combo[1]
    B_com = u.select_atoms(f'chainID {seg_combo[1]}').center_of_mass()
    # this identifies A's relationship to B. 
    comparison_dist = np.linalg.norm(A_pos-B_com)
    comparison_angle = np.rad2deg(get_angle(A_pos,com,B_com))
     
    for key, seg_list in identical_subunits.items():
        if seg_combo[0] in seg_list and seg_combo[1] in seg_list:
            A_group, B_group = key, key
        elif seg_combo[0] in seg_list and seg_combo[1] not in seg_list:
            A_group = key
        elif seg_combo[1] in seg_list and seg_combo[0] not in seg_list:
            B_group = key

    relationships = []
    for seg1 in identical_subunits[A_group]:
        distances = {}
        distance_difs = {}
        angle_difs = {}
        for seg2 in identical_subunits[B_group]:
            if seg1 != seg2:
            
                pos1 = u.select_atoms(f'chainID {seg1} and resnum {resid} and name CA').positions[0]
                pos2 = u.select_atoms(f'chainID {seg2}').center_of_mass()
                distance = np.linalg.norm(pos1-pos2)
                distances[(seg1,seg2)] = distance
                angle_difs[(seg1,seg2)] = np.abs(np.rad2deg(get_angle(pos1,com,pos2)) - comparison_angle)
                distance_difs[(seg1,seg2)] = np.abs(distance-comparison_dist)

        min_dist, min_angle = min(distance_difs, key=distance_difs.get), min(angle_difs, key=angle_difs.get)  
        if min_dist != min_angle:
            if ((distance_difs[min_dist]) + (angle_difs[min_dist])) < (distance_difs[min_angle]) + (angle_difs[min_angle]):
                relationships.append(tuple(sorted(min_dist)))
            else:
                relationships.append(tuple(sorted(min_angle)))
        else:       
            relationships.append(tuple(sorted(min_dist)))
    return relationships


def get_equivalent_interactions(representative_chains, u):
    '''
    For each chain in representative_chains, find all other identical chains' interaction partners
    that are equivalent to the representative_chains interactions with all the other chains in the protein.
    For instance, chain A's interaction with a subunit D on the other side of the protein might be equivalent to
    chain B's interaction with subunit E for a symmetric multimer.

    This is useful when averaging the contact frequencies of a multimer and determining the correct naming
    for the averaged contact record and to ensure it's depicted correctly when visualized.

    Parameters
    ----------
    representative_chains : list of strings
        A list with the names of the chain ids that you want to establish equivalent interactions for

    '''

    segids = u.segments.segids
    all_chain_dists = get_all_chain_dists(u)
    identical_subunits = find_identical_subunits(u)

    equivalent_interactions = {}
    for chain in [chain for chain in representative_chains]:
        for segid in segids:
            if segid != chain:
                equivalent_interactions[tuple(sorted((chain,segid)))] = asymmetric_measurements((chain,segid),identical_subunits,u, all_chain_dists)

    return equivalent_interactions

In [ ]:
sorted_distances ## The distances between priority_subunit and the other subunits
                # dictionary keys are integer or tuples for inter-subunit data
segids  # the list of all the segids
sorted_all_chain_dists ## the minimum distances between the segid key and all other segments 
                # the other segments are in a nested dictionary holding their distances as values
partner_chains # the dictionary of segid keys and lists of all other segids it makes contact with from the contact df
identical_subunits # the dictionary containing lists of identical subunit segids
#angles # the angles between a list of contacting residues using the com as the vertex
#distances # the distances between a list of contacts 

# Averageing Loop

In [126]:
df_copy = df.copy()
contacts = df.columns
# hold the averaged data
averaged_data = {}
# select one of each type of chain to be the representative chain for average naming
representative_chains = ['A', 'G']
# determine what the equivalent chain interactions relative to representative chains are for all the subunits
equivalent_interactions = get_equivalent_interactions(representative_chains,u)
# Retrieve/assign these from sorted identical subunits first indices.... although we want the other identical subunits first index to neighbor the chain in
# the first set... A needs to be next to G... This is the case for ATCASE - can just edit the chains in the PDB.... and produce a PDB for visualization along with averaging...
# identify priority name from each set of identical subunits

starting_length = len(df_copy.columns)

while len(df_copy.columns) > 0:
     resids = _parse_id(df_copy.columns[0])

     # find all of the other contacts that involve these residue names and numbers
     # intersubunit contacts can have swapped resids
     # so search with both regexes 
     regex1 = f"[A-Z1-9]+:{resids['resna']}:{resids['resida']}(?!\d)-[A-Z1-9]+:{resids['resnb']}:{resids['residb']}(?!\d)"
     regex2 = f"[A-Z1-9]+:{resids['resnb']}:{resids['residb']}(?!\d)-[A-Z1-9]+:{resids['resna']}:{resids['resida']}(?!\d)"
     regex = f"{regex1}|{regex2}"
     to_average = list(df_copy.filter(regex=regex, axis=1).columns)
     # now filter these to ensure you're only taking ones with equivalent_interaction names
     chaina, chainb = resids['chaina'], resids['chainb']




     if chaina == chainb:
          identical_pair = True
          for key, identical_subunit_list in identical_subunits.items():
               if chaina in identical_subunit_list:
                    for representative_chain in representative_chains:
                         if representative_chain in representative_chains:
                              representative_pair = (representative_chain, representative_chain)
                              
     # determine which equivalent_interaction set it came from 
     else:
          paira = (chaina,chainb)
          #pairb = (chainb,chaina)
          for representative_pair_name, equivalent_interaction_list in equivalent_interactions.items():
               for pair in equivalent_interaction_list:
                    if paira == pair: #or pairb == pair:
                         representative_pair = representative_pair_name
                         matching_pair = pair
                         break

     

     # make is_identical_subunit function
     # Can't deal the same with things ivolving a single subunit because not accounting for it in equivalent_interactions
     if identical_pair:
          averaged_name =  f"{representative_pair[0]}:{resids['resna']}:{resids['resida']}-{representative_pair[1]}:{resids['resnb']}:{resids['residb']}"
          to_drop = []
          for contact_name in to_average:
               contact_info = _parse_id(contact_name)
               # the chain equivalency check isn't necessary
               # but if if the chaina variable isn't in the same identical subunit group as the current contact's chain
               # drop it (that means it's happens to have the same resname and resnum but is happening on a different kind of subunit)
               if (contact_info['chaina'] != contact_info['chainb']) or \
               (contact_info['chaina'] not in identical_subunits[get_chain_group(chaina, identical_subunits)]):
                    to_drop.append(contact_name)
          for contact_name in to_drop:
               to_average.remove(contact_name)

     else:
          # if they're not identical and the first representative pair member matches is from the same identical subunit set as chaina
          # this only works for inter-hetero-subunit contacts NOT for inter-homo-subunit

          # inter-homo-subunit then need to first drop everything that involves the hetero-subunit
          to_drop = []
          resnums = []
          matched_name = None
          flipped = None
          for contact_name in to_average:
               contact_info = _parse_id(contact_name)
               if (contact_info['chaina'],contact_info['chainb']) not in equivalent_interactions[representative_pair]:
                    to_drop.append(contact_name)
               else: 
                    # use this to identify the outlier that would give a flipped naming scheme
                    resnums.append(contact_info['resida'])
                    if (contact_info['resida'], contact_info['residb']) == representative_pair:
                         matched_name = contact_name
                    elif (contact_info['residb'], contact_info['resida']) == representative_pair:
                         flipped = True
          # if inter-hetero subunit     
          if get_chain_group(representative_chain[0], identical_subunits) != (get_chain_group(representative_chain[1], identical_subunits)):
               # and the hetero unit order matches (This shouldn't be happening in the test case)
               if get_chain_group(representative_chain[0], identical_subunits) ==  get_chain_group(resids['chaina'], identical_subunits):
                    averaged_name =  f"{representative_pair[0]}:{resids['resna']}:{resids['resida']}-{representative_pair[1]}:{resids['resnb']}:{resids['residb']}"
               else:
                    averaged_name =  f"{representative_pair[0]}:{resids['resnb']}:{resids['residb']}-{representative_pair[1]}:{resids['resna']}:{resids['resida']}"
          # it's inter-homo subunit
          else:
               # this should be the only case where you have to guess.. 
               # if the contact chains are the same as the representative pair, you eliminate one situation
               if matched_name is not None:
                    averaged_name = matched_name
               elif flipped is not None:
                    averaged_name = f"{representative_pair[0]}:{resids['resnb']}:{resids['residb']}-{representative_pair[1]}:{resids['resna']}:{resids['resida']}"
               else:
                    #compare_distances between the resids (and possibly all the others to get avg dist) and then the representative_pair and flipped representative_pair



          # drop the ones that don't fit the original pair
          for contact_name in to_drop:
               to_average.remove(contact_name)

         
          ##### Filter out ones that don't fit the equivalent_interaction of the resids[] data
          to_drop = []
          for contact_name in to_average:
               contact_info = _parse_id(contact_name)
               if (contact_info['chaina'],contact_info['chainb']) not in equivalent_interactions[representative_pair]:
                    to_drop.append(contact_name)
          for contact_name in to_drop:
               to_average.remove(contact_name)
     ## TODO adjust average to account for number of subunits
     #if chaina == chainb:
     #    denominator = len(identical_subunits(get_chain_group(chain)))
     #else:
     #  denominator = len(longest_identical_chain_group participating in intersubunit contact)
     averaged_data[averaged_name] = df_copy[to_average].sum(axis=1)/6
     df_copy.drop(to_average, axis=1, inplace=True)
     # report length of columns or percent remaining at intervals.
     if len(df_copy.columns)%10 == 0:
          print(len(df_copy.columns))

31530
31500


KeyboardInterrupt: 

In [127]:
to_average

[]

In [128]:
averaged_name

'G:LEU:46-J:ILE:42'

In [129]:
to_drop

['G:LEU:46-J:ILE:42',
 'H:ILE:42-K:LEU:46',
 'H:LEU:46-K:ILE:42',
 'G:ILE:42-J:LEU:46',
 'I:LEU:46-L:ILE:42',
 'I:ILE:42-L:LEU:46']

In [130]:
representative_pair[0] 

'G'

In [132]:
paira

('G', 'J')

In [133]:
representative_pair

('G', 'J')

In [134]:
resids

{'chaina': 'G',
 'resna': 'LEU',
 'resida': '46',
 'chainb': 'J',
 'resnb': 'ILE',
 'residb': '42'}